In [1]:
import scanpy as sc

import numpy as np
import pandas as pd
from anndata import AnnData
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
#import skimage
import seaborn as sns 
import tangram as tg
import diopy
import time
import os

sc.logging.print_header()


%matplotlib inline

D:\anaconda\envs\tangram-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.10.0 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 pynndescent==0.5.8


In [ ]:
123

In [ ]:
E:\百度云下载\h5\st

In [2]:

adata_sc=diopy.input.read_h5(file ='E:/百度云下载/h5/st/C2D2_CB_cellbin.h5')#这


In [3]:
adata_sc

AnnData object with n_obs × n_vars = 9197 × 25057
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'coor_x', 'coor_y', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.5', 'seurat_clusters', 'celltype'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [22]:
123

123

In [2]:


# 设置工作目录
working_dir = 'E:/doctor/自研/mapping/data/20230814final/h5/st/'
os.chdir(working_dir)

# 获取所有文件列表
datalist = [f for f in os.listdir() if os.path.isfile(f)]

# 循环遍历文件
for i in range(2, len(datalist)):
# for i in range(0, 1):
    print(datalist[i])

    # 读取st数据
    st_path = os.path.join(working_dir, datalist[i])
    st_obj = diopy.input.read_h5(st_path)
    
    # 根据st数据选取sc数据
    if datalist[i] in ["st_HIP_starmap.h5", "C2D2_HIP_cellbin.h5", "HIP_slide_seq.h5", "HIP_test1.h5"]:
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_HIP.h5")
        anno = 'annotation'

    elif datalist[i] == "C2D2_CB_cellbin.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_CB.h5")
        anno = 'annotation'

    elif datalist[i] == "C2D2_OB_cellbin.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_OB.h5")
        anno = 'type'

    elif datalist[i] == "st_human_CTX.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_human_CTX.h5")
        anno = 'Subclass'


    elif datalist[i] == "st_mouse_CTX.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_mouse_CTX.h5")
        anno = 'Type'
       
    #开始时间
    start_time = time.time()
    
    adata_sc=sc_obj
    adata_st=st_obj
    sc.tl.rank_genes_groups(adata_sc,groupby=anno,use_raw=False)
    markers_df=pd.DataFrame(adata_sc.uns["rank_genes_groups"]["names"]).iloc[0:100,:]
    markers=list(np.unique(markers_df.melt().value.values))
    #去检查overlap的基因是不是都在
    tg.pp_adatas(adata_sc,adata_st,genes=markers)
    ad_map = tg.map_cells_to_space(adata_sc,adata_st,
                              mode="cells",
                              density_prior='rna_count_based',
                              num_epochs=100,
                              device="cpu")
    tg.project_cell_annotations(ad_map,adata_st,annotation=anno)
    annotation_list=list(pd.unique(adata_sc.obs[anno]))

    print("saving \n")
    #重写一个名称
    name="E:/doctor/自研/mapping/data/20230814final/result/tangram/"+datalist[i]+"_tangram_prediction.csv"
    adata_st.obsm['tangram_ct_pred'].sort_index(axis=1).columns
    adata_st.obsm['tangram_ct_pred'].to_csv(name)
    
    end_time = time.time()
    time_diff = end_time-start_time
    print(time_diff)
    
    time_name="E:/doctor/自研/mapping/data/20230814final/result/tangram/"+datalist[i]+"_tangram_prediction_time.csv"
    pd.Series([time_diff], name='time_diff').to_csv(time_name, header=True)



C2D2_OB_cellbin.h5


INFO:root:1085 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:22503 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1085 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.187, KL reg: 0.000


KeyboardInterrupt: 

In [3]:
datalist[i]

'C2D2_OB_cellbin.h5'